# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    17-Jan-2026**

**News Source / Catalyst:** Canada and China slash tariffs as they mend ties after years of dispute

**Markets (s) Impacted:** FX, COM

**Trade Taxonomy:** RV Macro

**Trade Idea (1 sentence):**
> Long a volatility-weighted basket of CAD and CNH against USD to capture the easing of trade frictions and the "de-risking" of the Canadian economy from US-centric trade volatility.

**Client/Desk Pitch (30 seconds):**
> We are fading the "US-Canada-China" tension premium. As Canada slashes EV tariffs from 100% to 6.1% and China reciprocates on agriculture, the fundamental outlook for Canadian exports and Chinese industrial demand improves. We are shorting the USD as the "Trump-unbothered" stance suggests a temporary lull in US trade-war escalation, allowing these bilateral partners to recover.

**Why Now:**
> The Jan 16 meeting marks the first visit by a Canadian PM to Beijing in nearly a decade. The immediate implementation of canola tariff cuts (March 1) provides a concrete fundamental floor for the trade.

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
> The "Carney Pivot" transforms Canada from a US satellite economy into a diversified global exporter. This reduces the risk premium on CAD. Simultaneously, CNH benefits from a rare "break" in the Western alliance's unified front on EV tariffs, providing a relief rally for Chinese industrials.

**Long Leg (Tickers + Rationale):**
- CAD (CADUSD=X), CNH (CNYUSD=X): CAD benefits from $5bn canola sector relief; CNH benefits from EV market access

**Short Leg (Tickers + Rationale):**
- USD (DX-Y.NYB or UUP): USD acts as the funding leg and represents the "single trade partner" reliance that Canada is actively exiting

**Weights:** Vol-weight
- CAD is significantly more volatile than CNH; CNH requires a higher notional to match CAD’s daily move.

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        S          |       |
| Oil Beta             |        M          |       |
| Rates Duration       |        W          |       |
| Growth vs Value      |        M          |       |
| Risk-On/Off          |        M          |       |
| China/EM Beta        |        S          |       |
| Volatility Regime    |        M          |       |

**Catalyst Type:** Policy / regime shift

**Realisation Window:** 4-8 weeks (leading up to March 1 tariff implementation)

**Holding Period Selected:**  20 days

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** 2.5% of AUM per leg.

**Stop-Loss:** -2.2%

**Take-Profit (optional):** 5.0%

**Re-entry:** Yes, if USD strength is purely driven by Fed hawishness rather than trade-war escalation

**FX-Hedging Required:** No (This is a pure FX cross-play)

**Liquidity Notes:** CAD and CNY (Onshore) are highly liquid; execution slippage should be minimal during NY/London overlap.

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [1]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [49]:
# --- Parameters ---

long_tickers = ["CADUSD=X", "CNYUSD=X"]  
short_tickers = ["DX-Y.NYB"]            
start_date = "2020-01-01"                
holding_period = 20                      
stop_loss = -0.022                       
take_profit = 0.055                                            

# --- Weightings ---

weights_long = 1 / len(long_tickers)
weights_short = 1 / len(short_tickers)


In [50]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()

data


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_38891/1853824072.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  3 of 3 completed


Ticker,CADUSD=X,CNYUSD=X,DX-Y.NYB
Date,,,
2020-01-01,0.7691,0.1436,NaN
2020-01-02,0.7708,0.1436,96.8500
2020-01-03,0.7702,0.1436,96.8400
2020-01-06,0.7700,0.1436,96.6700
2020-01-07,0.7714,0.1434,96.9800
...,...,...,...
2026-01-13,0.7207,0.1434,99.1300
2026-01-14,0.7201,0.1433,99.1300
2026-01-15,0.7202,0.1434,99.3200


In [51]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [52]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
73,2025-08-12,0.0111
74,2025-09-09,-0.0107
75,2025-10-07,-0.0194
76,2025-11-04,0.0091
77,2025-12-02,0.0272


In [53]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         78.0000
Win Rate              0.4359
Average PnL           0.0010
Best Trade            0.0838
Worst Trade          -0.0336
Stop-Out Frequency    0.3333
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 78
- Win Rate: 43.59%
- Avg PnL: +0.0010
- Best: +0.0838
- Worst: –0.0336
- Stop-Outs: 0.3333

**What went right:**
- Trade détente periods tend to reward CAD/CNH vs USD, reflected in high best-trade outliers
- CNYUSD trades with low vol, lowering stop-outs and providing stabilisation 
- DXY weakness windows (2020, 2023) aligned well with the thesis
- Stop-loss (-2.2%) is properly calibrated for FX RV environments, unlike equity-vol trades
- The macro intuition (fade US-centric trade risk) is validated by historical factor behaviour

**What went wrong:**
- DXY composition issue: the leg is 57% EUR, so EUR cycles dominated the backtest rather than CAD/CNH fundamentals
- CNYUSD=X is not CNH: onshore CNY volatility is heavily managed by PBOC and doesn’t behave like true FX beta
- Trade relies on geopolitics, which the historical window (2020–25) does not fully reflect
- CAD sensitivity to oil introduces an unwanted commodity factor
- Duration too short (20 days) for trade détente themes, which typically play over multi-quarter horizons

**Factor that actually drove results:**
1. USD broad factor (DXY)
2. Oil beta (via CAD)
3. EM beta (via CNY’s correlation)
4. EUR cycle (via DXY weighting)

So, the trade is actually more similar to a CAD/CNY EM basket vs USD factor trade.

**Would I trade this live?**
> Yes - but with modified instruments. The forward macro setup is fundamentally bullish (ie Trade détente narrative boosts CAD & CNH, China gets non-US import channels, Canada diversifies export base, USD loses its trade-war bid). I would prefer EURUSD leg instead of DXY to eliminate composition bias, or USD/CNH and USD/CAD directly for cleaner expression.

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [54]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [55]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [56]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
73,RV-F07A8DF0,2026-01-17 12:45:21,Macro RV Spread,RV Macro Spread,"CADUSD=X, CNYUSD=X",DX-Y.NYB,2025-08-12,2025-09-08,-97.6674,-97.0185,1.1147,-2.2000,5.5000,CLOSED,27,
74,RV-55FDCBDA,2026-01-17 12:45:21,Macro RV Spread,RV Macro Spread,"CADUSD=X, CNYUSD=X",DX-Y.NYB,2025-09-09,2025-10-06,-97.3574,-97.6817,-1.0735,-2.2000,5.5000,CLOSED,27,
75,RV-BB6B6284,2026-01-17 12:45:21,Macro RV Spread,RV Macro Spread,"CADUSD=X, CNYUSD=X",DX-Y.NYB,2025-10-07,2025-11-03,-98.1512,-99.4430,-1.9403,-2.2000,5.5000,CLOSED,27,
76,RV-2F9895CF,2026-01-17 12:45:21,Macro RV Spread,RV Macro Spread,"CADUSD=X, CNYUSD=X",DX-Y.NYB,2025-11-04,2025-12-01,-99.7945,-98.9814,0.9144,-2.2000,5.5000,CLOSED,27,
77,RV-5D5EBB62,2026-01-17 12:45:21,Macro RV Spread,RV Macro Spread,"CADUSD=X, CNYUSD=X",DX-Y.NYB,2025-12-02,2025-12-30,-98.9322,-97.8033,2.7239,-2.2000,5.5000,CLOSED,28,


In [57]:
blotter.to_csv("trade_blotter.csv", index=False)